In [39]:
### '''Import packages'''

#import schedule
import time
import numpy as np
import pandas as pd
import urllib
import urllib.request
import requests
import re
from bs4 import BeautifulSoup
from html.parser import HTMLParser
from datetime import datetime
#from htmldate import find_date

###################################
'''Open URL and Beautiful Soup '''

urldata = "https://www.basketball-reference.com/friv/injuries.fcgi" 
#urldata = "https://www.basketball-reference.com/leagues/NBA_2018_per_game.html"
#urldata = "https://www.basketball-reference.com/leagues/NBA_2021_per_game.html"
r = requests.get(urldata)
soup_new = BeautifulSoup(r.text,'html.parser')
league_table = soup_new.find('table', class_='sortable stats_table')


'''File Name Path Details'''
now = datetime.now()
file_date = (now.strftime("%d_%b_%Y"))
file_date.replace(r"/",".")
#file_name = "NBA_Player_Stats2.csv"
file_name = "NBAInjuryReport_" + str(file_date) + '.csv'
###################################
'''Defined functions below.'''

def Source_check():
    global urldata
    a = urllib.request.urlopen(urldata)
    if a.getcode() == 200:
        print("Result code: " + str(a.getcode()))
    else:
        print("Error, cannot parse results")
        
def Table_name():
    Title = soup_new.find('title')
    Title = re.search('<title>(.*?)</title>', str(Title)).group(1)
    print(Title)

def url():
    for link in soup_new.find_all('a'):
        print(link.get('href'))
        
def Header_row():
	head = league_table.find('thead')
	col = []
	column_name = re.findall('label="(.+?)"', str(head))
	st = column_name
	for i in st:
		list_i = [i]
		col.extend(list_i)
	return(column_name)

def nba_player_data():
	i = 0
	nba_player_data_list = []
	for team in league_table.find_all('tbody'):
		rows = team.find_all('tr')
		for row in rows:
			if i < 700:
				i += 1
				nba_players = row.find_all('th') + row.find_all('td')
				for player in nba_players:
					nba_player_data_list.append(player)
	return(nba_player_data_list)

def actual_data():
	list_ = nba_player_data()
	max_list_count = len(list_)
	col = []
	for num in range (0,max_list_count,1):
		if num ==0 or num%4==0:
			name = re.search('csk="(.+?)"',str(list_[num]))
			if name:
				col.append(name.group(1))
		else:
			actual_value = re.search('">(.+?)<',str(list_[num]))
			if actual_value is not None:
				col.append(actual_value.group(1))
			else:
				col.append('0.0')
	return(col)

def list_partition():
	main = actual_data()
	start= 0
	int = 4
	variable_assign = ['x']
	tot = len(main)
	df = pd.DataFrame(columns = Header_row())
	while start < tot:
		for part in variable_assign:
			part = main[start:(start + int)]
			df_length = len(df)
			df.loc[df_length] = part
			break
		start += int
	return(df)

###################################
'''Test Functions'''
#list_partition()
#nba_player_data()
#Source_check()
#Header_row()
#Table_name()
#actual_data()
#print(len(actual_data()))
###################################

###################################
'''Dataframe (pandas)'''
df = list_partition() 
df['Description'] = df['Description'].str.replace(',','')
df['Team'] = df['Team'].str[16:19]
df['Update'] = df['Update'].str[-12:]
df['Update'] = df['Update'].astype('datetime64[ns]')
#df["Timestamp"] = datetime.datetime.now()
df = df.reset_index(drop=True)
df.to_csv(file_name)

###################################